# Lesson 4 - Dataset Pools

Make imports

In [ ]:
import os
import pandas as pd
import csv

from ansys.mapdl.core import MapdlPool

Create several onstances of mapdl

In [ ]:
# Initialize 3 instances of mapdl running on 8 processors in SMP mode
pool = MapdlPool(n_instances=3, nproc=8, additional_switches="-smp")

In [ ]:
# Set mapdl model as a function
def model(mapdl, load_x_inches, load_y_inches):
    """
    Get mapdl model as a function to interact with pool
    """
    # Set run counter
    run_number = 0

    run_number += 1

    print(f"Start model: {run_number}")

    # Define area length and width
    length_inches = 4.5
    height_inches = 2

    # Define position of sensors and first load as a dictionary
    sensors = {
        "x1": 1,
        "y1": 0.5,
        "x2": 2.25,
        "y2": 0.5,
        "x3": 3.5,
        "y3": 0.5,
        "x4": 1,
        "y4": 1.5,
        "x5": 2.25,
        "y5": 1.5,
        "x6": 3.5,
        "y6": 1.5,
        "x7": 0.5,
        "y7": 1,
        "x8": 4,
        "y8": 1,
        "load_x": load_x_inches,
        "load_y": load_y_inches,
    }

    # Set sensors data as empty dictionary
    sensors_data = {}

    # Create rectangular area
    # Clear model
    mapdl.clear()

    # Enter pre-processor
    mapdl.prep7()

    # Create a rectangular area or block volume by corner points
    mapdl.blc4(0, 0, length_inches, height_inches, 0)

    # Delete un-meshed areas
    mapdl.adele("all")

    # Generate a fillet line between two intersecting lines
    mapdl.lfillt(1, 2, 0.2)
    mapdl.lfillt(2, 3, 0.2)
    mapdl.lfillt(3, 4, 0.2)
    mapdl.lfillt(4, 1, 0.2)

    # Compress the numbering of defined items
    mapdl.numcmp("area")

    # Generate an area bounded by previously defined lines
    mapdl.al("all")

    # Define hard points by sensor coordinates
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        mapdl.hptcreate(
            type_="area",
            entity=1,
            nhp="",
            label="coord",
            val1=sensors["x" + str(points)],
            val2=sensors["y" + str(points)],
            val3=0,
        )

    # Define hard point by load coordinates
    mapdl.hptcreate("area", 1, "", "coord", sensors["load_x"], sensors["load_y"], 0)

    # Merges coincident or equivalently defined items
    mapdl.nummrg("kp")

    # Set material properties
    mapdl.mp("ex", 1, 1e7)

    # Set element type
    mapdl.et(1, 181)

    # Set section type (shell representation)
    mapdl.sectype(1, "shell")

    # Set shell thickness
    mapdl.secdata(0.1, 1)

    # Specify the element shape as 2D
    mapdl.mshape(1, "2d")

    # Specify the element size to be meshed onto areas
    mapdl.smrtsize(3)

    # Generate nodes and area elements within areas
    mapdl.amesh("all")

    # Loop over nodes where sensors located
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )

        # Refine the mesh around specified nodes
        mapdl.nrefine(node, level=2, depth=1)

    # Unselect nodes
    mapdl.nsel("none")

    # Set component with selected nodes
    mapdl.cm("sensors", "node")

    # Select all entities
    mapdl.allsel()

    # Loop over nodes where sensors located again
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        # Get node
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )

        # Select a new set named "sensors"
        mapdl.cmsel("s", "sensors")

        # Additionally select current node
        mapdl.nsel("a", "node", "", node)

        # Group components into new component
        mapdl.cm("sensors", "node")

        # Select all entities
        mapdl.allsel()

    # Exit normally from a processor
    mapdl.finish()

    # Enter solution settings
    mapdl.slashsolu()

    # Define DOF constraints on lines???
    mapdl.dl("all", "", "all", 0)

    # Get node to apply force
    force_node = mapdl.queries.node(sensors["load_x"], sensors["load_y"], 0)

    # Apply force to node
    mapdl.f(force_node, "fz", 1)

    # Set solution as non-linear
    mapdl.nlgeom("on")

    # Solve solution
    mapdl.solve()
    # solve = mapdl.solve()

    # Exit normally from a processor
    mapdl.finish()
    # Enter post-processor
    mapdl.post1()

    # Set last time step/set as current
    mapdl.set("last")

    # Select all entities
    mapdl.allsel()

    # Set result as empty dictionary
    result = {}

    # Fill dictionary with load values
    result["Load_X"] = sensors["load_x"]
    result["Load_Y"] = sensors["load_y"]

    # Loop over nodes of sensors
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        # Set node
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )

        # Get x-component of total strain in node
        strain_x = mapdl.get_value("node", node, "epto", "x")

        # Get y-component of total strain in node
        strain_y = mapdl.get_value("node", node, "epto", "y")

        # Write data to result dictionary
        result["Strn_X_" + str(points)] = strain_x
        result["Strn_Y_" + str(points)] = strain_y

    print("End run")

    return result

Gether results

In [ ]:
# Set input data as empty list
inputs = []

# Set path to file
result_path = r"tmp\point_set.csv"

# Set path to save data
save_path = os.path.join(os.path.dirname(os.getcwd()), result_path)

save_path

In [ ]:
# Open the file in read mode and create a CSV reader object
with open(save_path, newline="") as csvfile:
    point_reader = csv.reader(csvfile, delimiter=",", quotechar="|")

    # Loop over each row in the CSV file
    for point in point_reader:
        # Convert each point to a tuple of floats and add it to the inputs list
        inputs.append((float(point[0]), float(point[1])))

In [ ]:
# # Write strain data to file
# with open(file=save_path, newline="", mode='r') as csvfile:
#     point_input = csv.writer(csvfile, delimiter=",", quotechar="|")
#
#     # Loop over
#     for point in point_input:
#         inputs += [(float(point[0]), float(point[1]))]

In [ ]:
# Set header for dataframe object
header = {
    "Load_X": [],
    "Load_Y": [],
    "Strn_X_1": [],
    "Strn_Y_1": [],
    "Strn_X_2": [],
    "Strn_Y_2": [],
    "Strn_X_3": [],
    "Strn_Y_3": [],
    "Strn_X_4": [],
    "Strn_Y_4": [],
    "Strn_X_5": [],
    "Strn_Y_5": [],
    "Strn_X_6": [],
    "Strn_Y_6": [],
    "Strn_X_7": [],
    "Strn_Y_7": [],
    "Strn_X_8": [],
    "Strn_Y_8": [],
}

In [ ]:
# Set dataset with only header
result_dataset = pd.DataFrame(data=header)

result_dataset

In [ ]:
print("Start solution -------")

# Run a function for each instance of mapdl within the pool, get list of function results
output = pool.map(func=model, iterable=inputs, progress_bar=True, wait=True)

# Set dummy dataset with only output
dummy_dataset = pd.DataFrame(data=output)

# Concatenate datasets to get dataset with results
result_dataset = pd.concat([result_dataset, dummy_dataset], ignore_index=True)

print("End solution")

# Show dataset
result_dataset

Serialize result file

In [ ]:
# # Set path to file
# result_path = r"tmp\point_set.csv"
#
# # Set path to save data
# save_path = os.path.join(os.path.dirname(os.getcwd()), result_path)
#
# # Serialize result dataset to file
# result_dataset.to_pickle(save_path)
#
# # Exit pool
# pool.exit()